# *SQL e Python no Mundo do Futebol e suas possíveis aplicações*

## <u>Com Python...<u/>

Ainda mantendo a ideia de uma empresa que vende relatórios de inteligência, utilizamos um novo dataset para a aplicação do algoritmo de naive-bayes (Multinomial), que irá funcionar como um classificador de sentimentos através de tweets de torcidas organizadas.

##### O que é o algoritmo de Nayve-bayes?

Naive Bayes usa um método similar para prever a probabilidade de classe diferente com base em vários atributos. Este algoritmo é usado principalmente em classificação de texto e com os problemas que têm múltiplas classes. O algoritmo utiliza a seguinte fórmula:
![](Images/Nayve-Bayes1.JPG)

#### Importando todas as bibliotecas necessárias

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pandas as pd
import joblib

### Aplicando o algoritmo de Naive-Bayes(Multinomial)

#### Leitura do dataframe

In [2]:
df_tw = pd.read_csv("torcida2.csv", delimiter=';')
df_tw

,Time,Torcida Organizada,Data,Hora,Tweet,Positivo / Negativo,Conta
0,Palmeiras,Mancha Verde,21/06/2021,00:43,"E ai, @DpaulaPatrick ? As 14h17 recebemos essa...",Negativo,torcidamv83
1,Palmeiras,Mancha Verde,ind,ind,Ainda bem que desde a ultima tempo os jogadore...,Positivo,ind
2,Palmeiras,Mancha Verde,ind,ind,"Em questao de salarios, o palmeiras esta conse...",Positivo,ind
3,Palmeiras,TUP,15/05/2021,22:02,E foda esses muleques que so pensam em TikTok ...,Negativo,tupminhavida
4,Palmeiras,TUP,ind,ind,Impressionante como o Bananotte e o Barros sao...,Negativo,ind
...,...,...,...,...,...,...,...
68,Ceara,Cearamor,12/02/2021,22:13,VAMO VOZAO!!!!!,Positivo,LongwellBelle
69,Coritiba,Imperio Alviverde,14/06/2021,01:42,"Os furachao fala, fala e fala, mas nunca consi...",Negativo,Iaanziito
70,Athletico Paranaense,Ultras do Athletico,16/01/2021,02:01,"O maior do sul nao tem como, os coxa nunca ser...",Negativo,FelipeAbel22
71,Athletico Paranaense,Ultras do Athletico,17/01/2021,03:01,sdds do time de 2019 e do tiago nunes,Positivo,cainhojatoba


#### Substituindo as classes finais (Positivo por 1; Negativo por 0) 

In [3]:
df_tw = df_tw.replace('Negativo', 0)
df_tw = df_tw.replace('Positivo', 1)
df_tw

,Time,Torcida Organizada,Data,Hora,Tweet,Positivo / Negativo,Conta
0,Palmeiras,Mancha Verde,21/06/2021,00:43,"E ai, @DpaulaPatrick ? As 14h17 recebemos essa...",0,torcidamv83
1,Palmeiras,Mancha Verde,ind,ind,Ainda bem que desde a ultima tempo os jogadore...,1,ind
2,Palmeiras,Mancha Verde,ind,ind,"Em questao de salarios, o palmeiras esta conse...",1,ind
3,Palmeiras,TUP,15/05/2021,22:02,E foda esses muleques que so pensam em TikTok ...,0,tupminhavida
4,Palmeiras,TUP,ind,ind,Impressionante como o Bananotte e o Barros sao...,0,ind
...,...,...,...,...,...,...,...
68,Ceara,Cearamor,12/02/2021,22:13,VAMO VOZAO!!!!!,1,LongwellBelle
69,Coritiba,Imperio Alviverde,14/06/2021,01:42,"Os furachao fala, fala e fala, mas nunca consi...",0,Iaanziito
70,Athletico Paranaense,Ultras do Athletico,16/01/2021,02:01,"O maior do sul nao tem como, os coxa nunca ser...",0,FelipeAbel22
71,Athletico Paranaense,Ultras do Athletico,17/01/2021,03:01,sdds do time de 2019 e do tiago nunes,1,cainhojatoba


In [4]:
# Corpus = Todos os tweets reunidos em um array
corpus = df_tw['Tweet'].values
corpus

array(['E ai, @DpaulaPatrick ? As 14h17 recebemos essa mensagem diretamente de voce em nosso Instagram. Agora a noite voce estava em balada, tirando com a cara do torcedor palmeirense. E agora, Mauricio Banana? O que mais falta pra virar homem e assumir o comando?',
       'Ainda bem que desde a ultima tempo os jogadores da base vem sendo mais utilizados',
       'Em questao de salarios, o palmeiras esta conseguindo pagar tudo em dia',
       'E foda esses muleques que so pensam em TikTok e nao querem saber de jogar mais',
       'Impressionante como o Bananotte e o Barros sao uns idiotas para contratar, nunca vi piores que eles',
       'Depois da triplice coroa esses jogadores nao querem saber de mais nada, serao cobrados no CT amanha as 10:00, vai quem quiser tirar satisfacao tambem',
       'a TUP nunca abaixa a cabeca nas brigas',
       'Gracas aos titulos desse ano a diretoria conseguiu ficar sem demitir ninguem durante a pandemia',
       'Certeza que os jogadores nao treinam p

In [5]:
# CountVectorizer() = Converte uma coleção de documentos de texto em uma matriz de contagens de tokens
vectorizer = CountVectorizer()

# fit_transform() = Ajustar aos dados e, em seguida, transformá-los
X_vec = vectorizer.fit_transform(corpus)

# Imprimindo cada palavra separadamente
print(vectorizer.get_feature_names())

['00', '10', '12', '14h17', '20', '2019', '21', 'abaixa', 'abrindo', 'acabando', 'aceitaram', 'acho', 'acompanhamos', 'adoramos', 'ae', 'agora', 'ai', 'aiai', 'ainda', 'alegria', 'alves', 'alvinegra', 'amanha', 'amo', 'ano', 'ao', 'aos', 'apaixonada', 'apanham', 'apequenou', 'apoiando', 'aqui', 'arranjam', 'as', 'assim', 'assistir', 'assumir', 'ate', 'atletico', 'azul', 'bahia', 'bahiaa', 'balada', 'banana', 'bananotte', 'bando', 'bar', 'barra', 'barros', 'base', 'bastante', 'bater', 'bem', 'benitez', 'besteira', 'blusa', 'bonita', 'bora', 'botafogo', 'branca', 'brasil', 'briga', 'brigar', 'brigas', 'busca', 'cabeca', 'cada', 'camisa', 'campeao', 'campeoes', 'campo', 'cancer', 'cap', 'cara', 'caras', 'certeza', 'certo', 'chamam', 'chance', 'cholorados', 'chora', 'chorar', 'chupaaa', 'classico', 'cobrados', 'cobrinhas', 'coelho', 'colinaaaaaaa', 'com', 'comando', 'comemorar', 'como', 'complicado', 'compromissado', 'condominio', 'conseguindo', 'conseguiu', 'consiguirao', 'contra', 'contr

In [6]:
# Atributos a serem considerados
X = X_vec.toarray()# Transformando em array
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
# Classe final
y = df_tw['Positivo / Negativo'].values # Pegando os valores da coluna do df
y1 = y.tolist() # Transformando em uma lista
print(y1)

[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0]


In [8]:
# Nayve-Bayes Multinomial
mult = MultinomialNB()

In [9]:
# Separando os X's e Y's para teste e treino (30% dos dados serão utilizados para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [10]:
# Normalização dos valores X_train e y_train com a função 'fit()'
mult.fit(X_train,y_train)

MultinomialNB()

In [11]:
# Valores preditos
y_pred = mult.predict(X_test)
y_pred

array([0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1],
      dtype=int64)

In [12]:
# Valores testados
y_test

array([0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0],
      dtype=int64)

### Avaliando o algoritmo de Naive-Bayes(Multinomial)

In [13]:
print(classification_report(y_test, y_pred, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.80      0.62      0.70        13
           1       0.58      0.78      0.67         9

    accuracy                           0.68        22
   macro avg       0.69      0.70      0.68        22
weighted avg       0.71      0.68      0.68        22



![](Images/metricas.JPG)

In [14]:
confusion_matrix(y_test, y_pred)

array([[8, 5],
       [2, 7]], dtype=int64)

![](Images/Matriz1.JPG)
![](Images/Matriz2.JPG)

Das 10 pessoas da amostra que fizeram tweets negativos, 8 foram previstas corretamente, enquanto 2 foram preditas tweetando positivamente. Já das 12 pessoas da amostra que fizeram tweets positivos, 7 foram previstas corretamente, enquanto 5 foram preditas tweetando negativamente.

### Persistindo o algoritmo de Naive-Bayes(Multinomial)

In [15]:
arq_nai = 'NAI_SocioTorcedor.sav'
joblib.dump(mult,arq_nai)

['NAI_SocioTorcedor.sav']

In [16]:
nai_carregado = joblib.load(arq_nai)
resultado = nai_carregado.score(X_test, y_test)
print(resultado)

0.6818181818181818


#### ACURÁCIA FINAL = 68,1%